## 9.2 원하는 숫자 생성하기

숫자를 무작위로 생성하지 않고 원하는 숫자를 지정해 생성하는 모델

다양한 방법이 있지만, noise에 label data를 hint로 넣어 주는 방법 사용!

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28*28
n_noise = 128
n_class = 10

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

Y값 추가 : 결과값 판정용은 아님, noise와 실제 이미지에 각각에 해당하는 숫자를 힌트로 넣어주는 용도로 사용!

In [4]:
def generator(noise, labels):
    with tf.variable_scope('generator'):
        inputs = tf.concat([noise, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation = tf.nn.relu)
        output = tf.layers.dense(hidden, n_input, activation = tf.nn.sigmoid)
        
    return output

### Generator NN 구성

* 변수 선언하지 않고 layers 사용

GAN : gen 과 dis 동시에 학습 -> 학습 시 각 신경망의 변수들을 따로따로 학습! 

-> tf.layers 사용하면 변수 선언하지 않고 tf.variable_scope 이용해 scope를 지정 가능! -> 나중에 이 scope에 해당하는 변수만 따로 불러올 수 있음

tf.concat 이용 -> noise 값에 labels 정보를 추가

tf.layers.dense 이용 -> hidden 만들고, 진짜 이미지와 같은 크기의 값을 만드는 output 구성

In [5]:
def discriminator(inputs, labels, reuse = None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
        
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation = tf.nn.relu)
        output = tf.layers.dense(hidden, 1, activation = None)
    
    return output

### Discriminator NN 구성

Generator과 같은 방법으로 만듦! -> dis는 진짜 이미지를 판별할 때와 가짜 이미지를 판별할 때 똑같은 변수 사용해야 함!

scope.reuse_variables 함수를 사용하여 이전에 사용한 변수를 재사용하도록 작성!

-> 출력값에 활성화 함수를 사용하지 않았음 => 손실값 계산에 sigmoid_cross_entropy_with_logtis 함수를 사용하기 위해!

In [6]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size = [batch_size, n_noise])

noise 생성 유틸리티 함수에서 noise를 균등분포로 생성!

In [7]:
G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

Generator 구성하고, 진짜 이미지 와 gen이 만든 이미지를 이용하는 dis를 하나씩 만들어 준다! 

gen에는 label 정보를 추가하여, 후에 label 정보에 해당하는 이미지를 생성하도록 유도! (Y 추가한 것의미)

가짜 이미지 구분자를 만들 때는 진짜 이미지 구분자에서 사용한 변수들을 재사용하도록 resue 옵션을 True로 설정!

In [8]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_real, labels = tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_gene, labels = tf.ones_like(D_gene)))

loss_D = loss_D_real + loss_D_gene

#### cost function 설정

원래는 진짜 이미지를 판별하는 D_real값은 1에 가까워지도록, 가짜 이미지를 판별하는 D_gene값은 0에 가까워지도록! 

-> but sigmoid_cross_entropy_with_logits 함수를 이용하면 코드를 더 간편하게 짤 수 있음!

loss_D = loss_D_real + loss_D_gene -> 이를 최소화 하면 dis를 학습시킬 수 있음

=> D_real은 1에 가까워야 하고(실제 이미지는 진짜라고 판별), D_gene은 0에 가까워야함(생성한 이미지는 가짜라고 판별)

-> loss_D_real : D_real의 결과값과 D_real의 크기만큼 1로 채운 값들을 비교(ones_like)

-> loss_D_gene : D_gene의 결과값과 d_gene의 크기만큼 0으로 채운 값들을 비교(zeros_like)

In [9]:
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_gene, labels = tf.ones_like(D_gene)))

loss_G 설정 -> gen(위조지폐범)을 학습시키기 위한 손실값

sigmoid_cross_entropy_with_logits 함수 이용하여 D_gene을 1에 가깝게 만드는 값을 손실값으로 취함!

In [10]:
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list = vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list = vars_G)

tf.get_collection 함수를 이용해 discriminator와 generator scope에서 사용된 변수들을 가져옴

-> 이를 최적화에 사용할 각 손실함수와 함께 최적화 함수에 넣어 학습 모델 구성!

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.trian.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batach):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict = {X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict = {Y: batch_ys, Z: noise})
        
    print('Epoch: ', '%04d' % epoch, 
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict = {Y: mnist.test.labels[:sample_size], Z: noise})
        
        fig, ax = plt.subplots(2, sample_size, figsize = (sample_size, 2))
        
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            
            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28,28)))
            ax[1][i].imshow(np.reshape(sampels[i], (28,28)))
            
        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)), bbox_inchoes = 'tight')
        plt.close(fig)
        

print('최적화 완료!')

앞의 GAN 모델과 거의 똑같 but Y의 입력값으로 batch_ys 값을 넣어줌! 

학습 중간중간에 gen으로 만든 이미지를 저장하는 코드를 작성

진짜 이미지와 비교해보기 위해 위쪽에는 진짜 이미지를 출력, 아래쪽에는 생성한 이미지를 출력!

- noise 가 무작위로 만들어짐 -> 매번 조금씩 다른 이미지 생성

- **생성된 이미지들은 mnist에 없는 새로운 손글씨 이미지!!!!!**